In [5]:
import os
from glob import glob
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from moviepy.editor import VideoFileClip


In [3]:
# Load video and 2D keypoints
# Setting
data_dir = r"/media/yiting/NewVolume/Data/Videos"
analysis_dir = r"/media/yiting/NewVolume/Analysis"
session_name = "2025-08-22"
camera_views = ["To", "TL", "TR", "BL", "BR"]

lp_dir = os.path.join(analysis_dir, session_name, "litpose")

In [51]:
# vid_path = os.path.join(lp_dir,"new_videos", "2025-08-22_09-02-11_camTo.mp4")
vid_path = os.path.join(lp_dir,"video_preds", "2025-08-22_09-02-11_camTo_labeled.mp4")
kp_2d_path = os.path.join(lp_dir, "video_preds", "2025-08-22_09-02-11_camTo.csv")
kp_2d = pd.read_csv(kp_2d_path, header=[0, 1, 2], index_col=0)
column_names = kp_2d.columns.tolist() # Get multiindex column names as list (model, feature, coord)

In [52]:
# selected_features = ['Small_Tip', 'Small_DIP', 'Small_PIP', 'Small_MCP']
selected_features = ['Index_Tip', 'Index_DIP', 'Index_PIP', 'Index_MCP']
feature_columns = [col for col in column_names if col[1] in selected_features]


In [53]:
def create_trace_frame(time_current, df_traces, feature_columns):
    """
    Generate a figure for the traces at a specific time point.
    Args:
        time_current (float): Current time in seconds.
        df_traces (pd.DataFrame): DataFrame containing the traces data.
        feature_columns (list): List of feature column multi-index tuples to plot.
    Returns:
        traces_frame (np.ndarray): Image array of the generated traces figure.
    """
    # Setting
    fs = 100  # Sampling frequency
    time_whole = np.arange(len(kp_2d)) / fs
    coordinates_to_plot = ['x', 'y', 'likelihood']  # Coordinates to plot
    feature_columns_indices = [df_traces.columns.get_loc(col) for col in feature_columns]

    # Create subplots for each coordinate
    fig, axs = plt.subplots(len(coordinates_to_plot), 1, figsize=(8, 6), sharex=True)

    for ax, cor in zip(axs, coordinates_to_plot):
        for feature, feature_idx in zip(feature_columns, feature_columns_indices):
            if feature[2] == cor:
                ax.plot(time_whole, kp_2d.iloc[:,feature_idx], label=feature[1])
        
        ax.axvline(x=time_current, color='black', linestyle='--', label='Current Time')  # Moving bar
        ax.set_xlabel('Time (s)')
        ax.set_ylabel(cor)
        if cor == 'likelihood':
            ax.set_ylim([-0.1, 1.1])
            ax.legend(fontsize = 6)
    fig.savefig('current_frame.png')
    traces_frame = cv2.imread('current_frame.png')
    traces_frame = cv2.cvtColor(traces_frame, cv2.COLOR_BGR2RGB)
    plt.close(fig)
    return traces_frame

In [54]:
# Visualization: video + feature traces across time 

def creat_combined_video(video_path, trace_path, output_path, feature_columns):

    # Load trace data
    trace_data = pd.read_csv(trace_path, header=[0, 1, 2], index_col=0)

    # Load video
    video = cv2.VideoCapture(video_path)
    fps = video.get(cv2.CAP_PROP_FPS)
    frame_count = int(video.get(cv2.CAP_PROP_FRAME_COUNT))
    frame_width = int(video.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))

    # Create the output video writer
    output_height = frame_height + 800  # Adjust for trace height
    output_width = frame_width
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out_video = cv2.VideoWriter(output_path, fourcc, fps, (output_width, output_height))

    # Process each frame
    for frame_idx in range(frame_count):
        ret, frame = video.read()
        if not ret:
            break

        # Calculate current time
        current_time = frame_idx / fps

        # Generate trace frame
        trace_frame = create_trace_frame(current_time, trace_data, feature_columns)

        # Combine video frame (upper) and trace frame (lower)
        combined_frame = np.zeros((output_height, output_width, 3), dtype=np.uint8)
        combined_frame[:frame_height, :, :] = frame
        combined_frame[frame_height:, :, :] = cv2.resize(trace_frame, (output_width, 800))

        # Write the frame
        out_video.write(combined_frame)

    # Release resources
    video.release()
    out_video.release()

    print(f"Output video saved to {output_path}")

In [55]:
output_path = os.path.join(lp_dir, "visualizations", "2025-08-22_09-02-11_camTo_indexFinger.mp4")
creat_combined_video(vid_path, kp_2d_path, output_path, feature_columns)

Output video saved to /media/yiting/NewVolume/Analysis/2025-08-22/litpose/visualizations/2025-08-22_09-02-11_camTo_indexFinger.mp4
